In [ ]:
import os
import cv2
import numpy as np

import glob
from keras._tf_keras.keras.preprocessing.image import img_to_array
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from mtcnn import MTCNN
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

In [4]:
# Load OpenCV's Haar Cascade Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


# Load MTCNN Face Detector
mtcnn_detector = MTCNN()

# Image Size
IMG_SIZE = (200, 200)


# Dataset Paths
folder_path = "D:/fold/Project/age_det/dataset/combined_faces/content/data"

h5_file = "dataset.h5"


In [5]:
def load_and_preprocess(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading image: {img_path}")
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype("float32") / 255.0
    return img_to_array(img)

In [6]:
def detect_and_crop_face(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Unable to read {img_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Try Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=4, minSize=(30, 30))

    
    if len(faces) == 0:
        # Try MTCNN
        mtcnn_faces = mtcnn_detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if len(mtcnn_faces) > 0:
            x, y, w, h = mtcnn_faces[0]['box']
        else:
            print(f"No face detected in {img_path}")
            return None
    else:
        x, y, w, h = faces[0]
    
    # Crop and resize face
    face = img[y:y+h, x:x+w]
    face = cv2.resize(face, IMG_SIZE)
    return face.astype("float32") / 255.0

In [7]:
# Load Images from Folder

# Load Images from Folder
def load_img_folder(folder_path):
    img_data = []
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder not found: {folder_path}")
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    if not image_files:
        print(" No images found in dataset folder!")
        return np.array([])
    
    for filename in image_files:
        img_path = os.path.join(folder_path, filename)
        img_array = load_and_preprocess(img_path)
        if img_array is not None:
            img_data.append(img_array)
    
    return np.array(img_data)

In [8]:
def age_accuracy(y_true, y_pred, threshold=5):
    """Returns the percentage of predictions within ±5 years of actual age"""
    correct = tf.abs(y_true - y_pred) <= threshold
    return tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
# Load Processed Dataset
X_train, y_train = [], []
image_paths = glob.glob(os.path.join(folder_path, "*.jpg"))

for img_path in image_paths:
    try:
        filename = os.path.basename(img_path)
        age = int(filename.split("_")[0])
        img = load_and_preprocess(img_path)
        if img is not None:
            X_train.append(img)
            y_train.append(age)
    except Exception as e:
        print(f" Skipping {img_path}: {e}")

X_train = np.array(X_train, dtype="float32")
y_train = np.array(y_train, dtype="float32")
print(f" Dataset Loaded: {X_train.shape[0]} images")


 Dataset Loaded: 33486 images


In [11]:
# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression output for age
])

d:\fold\Project\age_det\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Compile and Train Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', age_accuracy])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    17,334,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,428,033 (66.48 MB)

 Trainable params: 17,428,033 (66.48 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(X_train, y_train, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
419/419 ━━━━━━━━━━━━━━━━━━━━ 226s 532ms/step - age_accuracy: 0.2305 - loss: 265.4928 - mae: 12.4329 - val_age_accuracy: 0.0393 - val_loss: 896.7266 - val_mae: 26.6887
Epoch 2/40
419/419 ━━━━━━━━━━━━━━━━━━━━ 211s 503ms/step - age_accuracy: 0.2243 - loss: 106.1319 - mae: 7.8333 - val_age_accuracy: 0.1105 - val_loss: 902.2598 - val_mae: 25.7380
Epoch 3/40
419/419 ━━━━━━━━━━━━━━━━━━━━ 203s 484ms/step - age_accuracy: 0.2221 - loss: 87.4407 - mae: 7.0311 - val_age_accuracy: 0.1082 - val_loss: 674.6238 - val_mae: 22.0344
Epoch 4/40
419/419 ━━━━━━━━━━━━━━━━━━━━ 204s 488ms/step - age_accuracy: 0.2187 - loss: 77.3712 - mae: 6.5902 - val_age_accuracy: 0.1466 - val_loss: 722.5463 - val_mae: 22.6076
Epoch 5/40
419/419 ━━━━━━━━━━━━━━━━━━━━ 214s 510ms/step - age_accuracy: 0.2180 - loss: 67.4866 - mae: 6.1290 - val_age_accuracy: 0.1605 - val_loss: 498.9493 - val_mae: 18.3934
Epoch 6/40
419/419 ━━━━━━━━━━━━━━━━━━━━ 223s 531ms/step - age_accuracy: 0.2184 - loss: 61.3257 - mae: 5.8510 - val_ag

In [14]:
# Save Model
model.save("age_detection_model.h5")
print(" Model saved successfully!")

 Model saved successfully!
